<a href="https://colab.research.google.com/github/i0dheerajdileep/Emotion-Recognition-from-Facial-Expressions/blob/main/facialrecog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data_dir = '/content/drive/My Drive/face-recog/train'
test_data_dir = '/content/drive/My Drive/face-recog/test'


img_width, img_height = 48, 48
batch_size = 32


train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'


In [ ]:
model = keras.Sequential([
    # Convolutional layers
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),

    # Flatten the feature maps
    keras.layers.Flatten(),

    # Fully connected layers
    keras.layers.Dense(128, activation='relu'),

    # Output layer with softmax activation for multi-class classification
    keras.layers.Dense(7, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
epochs = 10 # Adjust the number of epochs as needed

history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
)

In [ ]:
import cv2

# img_width, img_height = 128, 128
emotion_labels = ['angry', 'happy', 'sad', 'surprised', 'neutral', 'disgusted', 'fearful']
test_image_paths = ['/content/1.jpg']

test_images = []

for image_path in test_image_paths:
    img = load_img(image_path, target_size=(img_width, img_height))
    img_gray = cv2.cvtColor(img_to_array(img), cv2.COLOR_BGR2GRAY)
    img_gray = img_gray / 255.0
    test_images.append(img_gray)

test_images = np.array(test_images)
test_images = np.reshape(test_images, (len(test_images), img_width, img_height, 1))

predictions = model.predict(test_images)

predicted_emotions = [emotion_labels[np.argmax(pred)] for pred in predictions]

print(predicted_emotions)